In [ ]:
#Correr solo si se va a plotear spectrogramas

import librosa

def plot_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", interpolation="nearest")

In [ ]:
import os
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary



In [ ]:
#librerias locales
#from ..libs import dataLoaders as mi_dataLoader
#from ..libs import clasificador_padre as mi_clasificador
from importlib.machinery import SourceFileLoader
mi_dataLoader = SourceFileLoader('dataLoadersV2', '../libs/dataLoadersV2.py').load_module()
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

# Cargando un DataSet

In [ ]:
root = '../'
Path_train = root + 'data/onlywoman_train.csv'
Path_val = root + 'data/onlywoman_val.csv'
Path_test = root + 'data/onlywoman_test.csv'

In [ ]:
testDf = pd.read_csv(Path_train)
testDf.keys()

In [ ]:
testDataSet = mi_dataLoader.clasification_dataset(testDf, file_col="fileMfcc", target_col="artist", seed = 98)

In [ ]:
sample, target = testDataSet[0:9]
sample[0].shape, target

# Generando el DataLoader

In [ ]:

#def collate_fn(batch):
#    _mels = torch.stack([item[0] for item in batch], dim = 0)
#    _targets = torch.stack([torch.tensor(item[1]) for item in batch], dim=0)
#    return _mels, _targets

test_dataloader = DataLoader(testDataSet, batch_size=100, shuffle=True)
muestra_data, muestra_target = next(iter(test_dataloader))
muestra_target

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(test_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
plot_spectrogram(train_features[0], train_labels[0])
#img = train_features[0].squeeze()
#label = train_labels[0]
#plt.imshow(img, cmap="gray")
#plt.show()
#print(f"Label: {label}")

# instanciando un modelo

In [ ]:
history = []

In [ ]:

clasificador = mi_clasificador.onlyWoman_MFCC_02(list(testDataSet.dictionary.keys()))
s = 0
#clasificador.architecture

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Hay {torch.cuda.device_count()} GPU(s) disponibles.")
    print(f"GPU activa: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA no está disponible, se utilizará la CPU.")
clasificador.to(device)

In [ ]:
def test_accur (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)

In [ ]:
    test_optimizer = torch.optim.Adam(clasificador.parameters(), lr=0.0001)
    test_criterion = torch.nn.CrossEntropyLoss()

In [ ]:
j = 1

In [ ]:
for i in range(10):
    loss_log, acur_log = clasificador.train_loop(1, test_dataloader, test_optimizer , test_criterion, test_accur, device)
    save_state = clasificador.state_dict()
    #torch.save(save_state, f"{root}modelos/temp/ow_2_{j}.pt")
    #mi_clasificador.save_model(clasificador,f"{root}modelos/temp/saveState_{i}.pt")
    history.append((loss_log, acur_log))
    j = j+1

In [ ]:
torch.save(history, f"{root}modelos/temp/history_ow_2_30epoch.pt")


In [ ]:
loss_log = history[1][0]
acur_log = history[1][1]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize = (14,14))

# Graficar en el primer subplot
axes[0].plot(range(len(acur_log)), acur_log, 'r-', label='Acuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Acur')
axes[0].legend()

axes[1].plot(range(len(acur_log)), loss_log, 'r-', label='Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

# Validando

In [ ]:
#Carga de las muestras y el DataLoader
#Path_val = './csv/validation_set.csv'
valDf = pd.read_csv(Path_val)
valDataSet = mi_dataLoader.clasification_dataset(valDf, file_col="fileMfcc", target_col="artist", seed = 98)
val_dataloader = DataLoader(valDataSet, batch_size=100, shuffle=False)


In [ ]:
j

In [ ]:
# Cargo el modelo
save_state = torch.load(root+f"/modelos/temp/ow_2_{j-1}.pt")
clasificador = mi_clasificador.onlyWoman_MFCC_02(list(valDataSet.dictionary.keys()))
clasificador.load_state_dict(save_state)

In [ ]:
#Separando las muestras y los targets
muestras = []
targets = []
for muestra_data, muestra_target in val_dataloader:
    muestras.append(muestra_data)
    targets.append(torch.tensor(muestra_target))
    print(len(muestras)*100,"/",len(valDataSet))
muestras[0].shape

In [ ]:
def acuracy_fn (prediccion, target_batch):
    return(torch.argmax(target_batch, axis=1) == torch.argmax(prediccion, axis=1)).sum().item() / len(target_batch)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn,device)

In [ ]:
resultado_evalucion #valid despues de cargar save state

In [ ]:
evalHisto.append(resultado_evalucion)

In [ ]:
evalHisto

# espacio para el train

In [ ]:
for l in tqdm(range(5)):
    for i in range(10):
        loss_log, acur_log = clasificador.train_loop(1, test_dataloader, test_optimizer , test_criterion, test_accur, device)
        save_state = clasificador.state_dict()
        #torch.save(save_state, f"{root}modelos/temp/ow_2_{j}.pt")
        #mi_clasificador.save_model(clasificador,f"{root}modelos/temp/saveState_{i}.pt")
        history.append((loss_log, acur_log))
        j = j+1
    torch.save(history, f"{root}modelos/temp/history_ow_2_{j-1}epoch.pt")
    resultado_evalucion = clasificador.evaluate(val_dataloader, loss_fn, acuracy_fn,device)
    evalHisto.append(resultado_evalucion)
    print(evalHisto)

In [ ]:
evalHisto

## Codigo usado mientras se generaba la arquitectura (osea, codigo desechable)

In [ ]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()

In [ ]:
muestra_data, muestra_target = next(iter(test_dataloader))
#muestra = torch.stack(muestra, 0)
muestra_data.shape

In [ ]:
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()
clasificador = mi_clasificador.onlyWoman_MFCC_01(list(testDataSet.dictionary.keys()))

In [ ]:
res =clasificador(muestra_data)
res.shape

In [ ]:
out = nn.Sequential(   # bloque que acondiciona la salida
    nn.Linear(*self.architecture[-1]), #Lineal que sale a las clases objetivo
    nn.Softmax( dim = 1))

In [ ]:
test_criterion(res,muestra_target)

In [ ]:
test_accur (res, muestra_target)

In [ ]:
#importlib.reload(mi_clasificador)
clasificador = mi_clasificador.Clasificador_01(testDataSet.dictionary.keys())
res =clasificador(muestra_data)
test_accur (res, muestra_target)


In [ ]:
(torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)).sum().item() / len(muestra_target)

In [ ]:
torch.argmax(muestra_target, axis = 1) == torch.argmax(res, axis=1)

In [ ]:
muestra_target.shape, res.shape

In [ ]:
torch.argmax(res, axis = 1)

In [ ]:
res

In [ ]:
torch.argmax(muestra_target, axis=1)

In [ ]:
muestra_target

In [ ]:
muestra_target == res

In [ ]:
res

In [ ]:
flat = nn.Flatten(start_dim = 0)
lin = nn.Linear(17920,3)
soft = nn.Softmax(dim = 0)

print("Red: ",res.shape)
aux = flat(res)
print("Flat: ",aux.shape)
aux = lin(aux)
print("Line: ",aux.shape)
aux = soft(aux)
print("Soft:",aux.shape)